# EE4375: Fifth Lab Session: Geometry and Mesh Generation using GMSH 

The **goals** of this fifth lab session of the [EE4375 Course](https://github.com/ziolai/finite_element_electrical_engineering/tree/main) is to explore the use of the [GMSH](https://gmsh.info) package for geometry and mesh generation. 

For the **geometry generation**, GMSH can be employed using one of three methods that are described next: 
1. The first method is to build the geometry in GMSH from the ground up using the GMSH primitives for points, lines, surfaces and volumes. This allows to generate 1D, 2D or 3D geometries. This way of working provides detailed control over the geometry definition. It allows to automate the geometry creation through scripting. Two examples are given later in this notebook.  A possible disadvantage is the large overhead for complex geometries. 
2. The second method is create the geometry in dedicated CAD modeling tools, to save the geometry to file and to import the geometry into the GMSH graphical user interface. Examples of public domain CAD modeling tools include [Blender](https://www.blender.org), [FreeCAD](https://www.freecad.org) and [Salome](https://www.salome-platform.org). 
3. The third method is to use GMSH as a plugin for CAD tools as e.g. a [plugin for Blender](https://github.com/blender-for-science/blendmsh), [plugin for FreeCAD](https://wiki.freecad.org/Macro_GMSH) or [plugin for SALOME](https://docs.salome-platform.org/latest/gui/GMSHPLUGIN/index.html). 

For the unstructured **mesh generation** in 2D and 3D, GMSH provides various algorithms that are described in the GMSH manual at [GMSH Mesh Module](https://gmsh.info/doc/texinfo/gmsh.html#Choosing-the-right-unstructured-algorithm). In 2D, a mesh consisting of triangles is called a [triangulation](https://en.wikipedia.org/wiki/Triangulation_(geometry)). Often a variant of the [Delaunay algorithm](https://en.wikipedia.org/wiki/Delaunay_triangulation) is used to construct these triangular meshes. We refer to [Mesh Generation](https://en.wikipedia.org/wiki/Mesh_generation) for more information on mesh generation. 

## Import Packages

In [3]:
try
    using Gmsh: Gmsh, gmsh
catch
    using gmsh
end 

using GR 
using LinearAlgebra
using SparseArrays 
using Plots

## Section 1: Unit Square Geometry  

The code that follows performs <b>geometry generation</b>, synchronizes the model, adds physical groups and performs <b>mesh generation</b> on the unit square geometry. 

First the <b>geometry</b> is generated in the following five steps:
1. four corner points of the square are defined. The points are labeled as 1 through 4. See docs of function addPoint() for more information;
2. four lines are defined as the edges of the square are defined by connecting previously defined points. Edges are formed by connecting points pairwise. The lines are given a start and end point. The lines are thus oriented. The edges are labeled as 1 through 4. See docs of function addLine() for more information;  
3. the boundary of the square is defined by a loop connecting the four edges. The orientation of the edges given an orientation to the loop. The loop is oriented such that an imaginary observer walking on the loop finds the domain on his left-hand side. The loop is labeled as 1. See docs of function addCurveLoop() for more information;   
4. the surface of the square is defined by the loop. It is on this square that the mesh generation will take place. This square is labeled as 1. See docs of function addPlaneSurface() for more information;

Next the CAD model is synchronized and physical groups are added to the model. 

Next the <b>mesh</b> on the geometry is defined by mesh generation. The mesh is generated by first meshing the four edges of the square. The mesh is subsequently propagated towards the interior of the square. The mesh density is controlled by the parameter lc. 

The mesh is optionally written to file and visualized using the GUI. 

<b>Exercises </b>: 
1. change the coordinate of one of the four points of the square and regenerate the mesh;
2. change the mesh density by changing the value of the parameter lc and regenerate the mesh. Apply different mesh density on one or more point or edges of the square; 
3. extend the code to the generation of a mesh on a pentagon; 
4. extend the code to the generation of a mesh on an L-shaped domain; 
5. extend the code to the generation of a mesh on a square with an inner square removed thus practising the orientation of the loops; 
6. share imagines of the meshes obtained of social media of choice and notify your favorite finite element instructor; 

In [5]:
#..1/7: initialize gmsh 
should_finalize = Gmsh.initialize()

#..2/7: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc = .1
#..define four points via (x,y,z) coordinates 
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
p2 = gmsh.model.geo.addPoint(1., 0,  0, lc, 2)
p3 = gmsh.model.geo.addPoint(1., 1., 0, lc, 3)
p4 = gmsh.model.geo.addPoint(0, 1., 0, lc, 4)
#..define four edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)
l2 = gmsh.model.geo.addLine(2, 3, 2)
l3 = gmsh.model.geo.addLine(3, 4, 3)
l4 = gmsh.model.geo.addLine(4, 1, 4)
#..define curved loop by connecting four edge labels  
loop = gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
#..define surface by curved loop 
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..3/7: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/7: assign physical groups
gmsh.model.addPhysicalGroup(1, [l1], -1, "bottom")
gmsh.model.addPhysicalGroup(1, [l2], -1, "right")
gmsh.model.addPhysicalGroup(1, [l3], -1, "top")
gmsh.model.addPhysicalGroup(1, [l4], -1, "left")
gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

#..5/7: generate two-dimensional mesh 
gmsh.model.mesh.generate(2)

#..6/7: write mesh to mesh and visualize the mesh  
#..if true, write mesh to file for further processing 
if (true) gmsh.write("data/square.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..7/7: finalize gmsh 
should_finalize && Gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000535916s, CPU 0.000486s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00428029s, CPU 0.004162s)
Info    : 142 nodes 286 elements
Info    : Writing 'data/square.msh'...
Info    : Done writing 'data/square.msh'


false

## Section 2:/ Read Mesh from file and perform loop over the elements 

The code that follows performs retrieval of information from the mesh. 
Note that 
1. given a 1D mesh, getElement(1) retrieves 1D elements
1. given a 2D mesh, getElement(1) retrieves 1D elements (boundary elements) and getElement(2) retrieves 2D element
1. given a 3D mesh, getElement(1) retrieves 1D elements (boundary elements) and getElement(2) retrieves 2D elements and getElement(3) to retrieve 3D elements;

In [9]:
#..1/4: Finalize gmsh
should_finalize = Gmsh.initialize()

#..2/4: Read mesh from file
gmsh.open("data/square.msh")

#..3/4: perform loop over the elemements 
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
nelements = length(element_ids[1])
  
for element_id in 1:nelements

    #....retrieve global numbering of the local nodes of the current element
    node1_id = element_connectivity[1][3*(element_id-1)+1]
    node2_id = element_connectivity[1][3*(element_id-1)+2]
    node3_id = element_connectivity[1][3*(element_id-1)+3]

    if (false)
      println("on element ", element_id, " node-1 has global number ", node1_id)
      println("on element ", element_id, " node-2 has global number ", node2_id)
      println("on element ", element_id, " node-3 has global number ", node3_id)
      println(" ")
    end 

end 

#..4/4: Finalize gmsh
should_finalize && Gmsh.finalize()

Info    : Reading 'data/square.msh'...
Info    : 9 entities
Info    : 142 nodes
Info    : 282 elements
Info    : Done reading 'data/square.msh'


false

## Section 3: Example: Inductor Geometry

Mesh generation for an inductor geometry.

In [11]:
#..1/5: initialize gmsh 
should_finalize = Gmsh.initialize()

#..2/5: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
#..define the model
model = gmsh.model
model.add("Inductor")
#..define mesh density near a point
lg = 0.7    #gap length in mm
lg = 5    #gap length in mm
mur = 2000;  #relative permeability of the core
mu0 = 4*pi*10^(-7);
curr_dens_value = 3.1085;
k = 0.2227
cl = 5     #mesh size at bounday
c2 = lg/2  #mesh size at airgap
c3 = 2     #mesh size at component
#..define four points in the geometry
gmsh.model.geo.addPoint(50, 50, 0., cl, 1)
gmsh.model.geo.addPoint(-50, 50, 0., cl, 2)
gmsh.model.geo.addPoint(-50, -50,0., cl, 3)
gmsh.model.geo.addPoint(50, -50, 0., cl, 4)
#..upper half of core
gmsh.model.geo.addPoint(-27.25, lg/2, 0., c2, 5)
gmsh.model.geo.addPoint(-27.25, 27.6+lg/2, 0., c3, 6)
gmsh.model.geo.addPoint(27.25, 27.6+lg/2, 0., c3, 7)
gmsh.model.geo.addPoint(27.25, lg/2, 0., c2, 8)
gmsh.model.geo.addPoint(20.6, lg/2, 0., c2, 9)
gmsh.model.geo.addPoint(20.6, 20.2+lg/2, 0., c3, 10)
gmsh.model.geo.addPoint(7.422, 20.2+lg/2, 0., c3, 11)
gmsh.model.geo.addPoint(7.422, lg/2, 0., c2, 12)
gmsh.model.geo.addPoint(-7.422, lg/2, 0., c2, 13)
gmsh.model.geo.addPoint(-7.422, 20.2+lg/2, 0., c3, 14)
gmsh.model.geo.addPoint(-20.6, 20.2+lg/2, 0., c3, 15)
gmsh.model.geo.addPoint(-20.6, lg/2, 0., c2, 16)
#..lower half of core
gmsh.model.geo.addPoint(-27.25, -lg/2, 0., c2, 17)
gmsh.model.geo.addPoint(-27.25, -27.6-lg/2, 0., c3, 18)
gmsh.model.geo.addPoint(27.25, -27.6-lg/2, 0., c3, 19)
gmsh.model.geo.addPoint(27.25, -lg/2, 0., c2, 20)
gmsh.model.geo.addPoint(20.6, -lg/2, 0., c2, 21)
gmsh.model.geo.addPoint(20.6, -20.2-lg/2, 0., c3, 22)
gmsh.model.geo.addPoint(7.422, -20.2-lg/2, 0., c3, 23)
gmsh.model.geo.addPoint(7.422, -lg/2, 0., c2, 24)
gmsh.model.geo.addPoint(-7.422, -lg/2, 0., c2, 25)
gmsh.model.geo.addPoint(-7.422, -20.2-lg/2, 0., c3, 26)
gmsh.model.geo.addPoint(-20.6, -20.2-lg/2, 0., c3, 27)
gmsh.model.geo.addPoint(-20.6, -lg/2, 0., c2, 28)
#..define four edges in the geometry
gmsh.model.geo.addLine(1, 2, 101)
gmsh.model.geo.addLine(2, 3, 102)
gmsh.model.geo.addLine(3, 4, 103)
gmsh.model.geo.addLine(4, 1, 104)
#..upper half of core
gmsh.model.geo.addLine(5, 6, 105)
gmsh.model.geo.addLine(6, 7, 106)
gmsh.model.geo.addLine(7, 8, 107)
gmsh.model.geo.addLine(8, 9, 108)
gmsh.model.geo.addLine(9, 10, 109)
gmsh.model.geo.addLine(10, 11, 110)
gmsh.model.geo.addLine(11, 12, 111)
gmsh.model.geo.addLine(12, 13, 112)
gmsh.model.geo.addLine(13, 14, 113)
gmsh.model.geo.addLine(14, 15, 114)
gmsh.model.geo.addLine(15, 16, 115)
gmsh.model.geo.addLine(16, 5, 116)
#..lower half of core
gmsh.model.geo.addLine(18, 17, 117)
gmsh.model.geo.addLine(19, 18, 118)
gmsh.model.geo.addLine(20, 19, 119)
gmsh.model.geo.addLine(21, 20, 120)
gmsh.model.geo.addLine(22, 21, 121)
gmsh.model.geo.addLine(23, 22, 122)
gmsh.model.geo.addLine(24, 23, 123)
gmsh.model.geo.addLine(25, 24, 124)
gmsh.model.geo.addLine(26, 25, 125)
gmsh.model.geo.addLine(27, 26, 126)
gmsh.model.geo.addLine(28, 27, 127)
gmsh.model.geo.addLine(17, 28, 128)
# close off winding slots (right to left)
gmsh.model.geo.addLine(21, 9, 129)
gmsh.model.geo.addLine(12, 24, 130)
gmsh.model.geo.addLine(25, 13, 131)
gmsh.model.geo.addLine(16, 28, 132)
# link core area to outside
gmsh.model.geo.addLine(2, 6, 133)
gmsh.model.geo.addLine(19, 4, 134)
gmsh.model.geo.addLine(9, 21, 135)
gmsh.model.geo.addLine(6, 2, 136)
gmsh.model.geo.addLine(4, 19, 137)
gmsh.model.geo.addLine(28,16, 138)
# redefine air gap lines for middle gap
gmsh.model.geo.addLine(24, 12, 139)
gmsh.model.geo.addLine(13, 25, 140)
#..define outer boundary
gmsh.model.geo.addCurveLoop([101, 133, 106, 107, 108, 135, 120, 119, 134, 104], 201) # air 1
gmsh.model.geo.addCurveLoop([102, 103, 137, 118, 117, 128, 138, 116, 105, 136], 202) # air 2
gmsh.model.geo.addCurveLoop([105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116], 203) # upper core
gmsh.model.geo.addCurveLoop([117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128], 204) # lower core
gmsh.model.geo.addCurveLoop([109, 110, 111, 130, 123, 122 , 121, 129], 205) # right winding gap
gmsh.model.geo.addCurveLoop([113, 114, 115, 131, 125, 126 ,127, 132], 206) # left winding gap
gmsh.model.geo.addCurveLoop([112, 140, 124, 139], 207) # middle air gap
#..define planar surface
gmsh.model.geo.addPlaneSurface([201], 301)
gmsh.model.geo.addPlaneSurface([202], 302)
gmsh.model.geo.addPlaneSurface([203], 303)
gmsh.model.geo.addPlaneSurface([204], 304)
gmsh.model.geo.addPlaneSurface([205], 305)
gmsh.model.geo.addPlaneSurface([206], 306)
gmsh.model.geo.addPlaneSurface([207], 307)
#..define physics
gmsh.model.addPhysicalGroup(2,[301; 302; 307],401)
gmsh.model.addPhysicalGroup(2,[303],402)
gmsh.model.addPhysicalGroup(2,[304],403)
gmsh.model.addPhysicalGroup(2,[305],404)
gmsh.model.addPhysicalGroup(2,[306],405)
#..synchronize the model 
gmsh.model.geo.synchronize()

#..3/5: generate two-dimensional mesh
model.mesh.generate(2)

#..4/5: write mesh to mesh and visualize the mesh  
#..if true, write mesh to file for further processing 
if (true) gmsh.write("data/inductor.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..5/5: finalize gmsh 
should_finalize && Gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 101 (Line)
Info    : [ 10%] Meshing curve 102 (Line)
Info    : [ 10%] Meshing curve 103 (Line)
Info    : [ 10%] Meshing curve 104 (Line)
Info    : [ 20%] Meshing curve 105 (Line)
Info    : [ 20%] Meshing curve 106 (Line)
Info    : [ 20%] Meshing curve 107 (Line)
Info    : [ 20%] Meshing curve 108 (Line)
Info    : [ 30%] Meshing curve 109 (Line)
Info    : [ 30%] Meshing curve 110 (Line)
Info    : [ 30%] Meshing curve 111 (Line)
Info    : [ 30%] Meshing curve 112 (Line)
Info    : [ 40%] Meshing curve 113 (Line)
Info    : [ 40%] Meshing curve 114 (Line)
Info    : [ 40%] Meshing curve 115 (Line)
Info    : [ 40%] Meshing curve 116 (Line)
Info    : [ 50%] Meshing curve 117 (Line)
Info    : [ 50%] Meshing curve 118 (Line)
Info    : [ 50%] Meshing curve 119 (Line)
Info    : [ 50%] Meshing curve 120 (Line)
Info    : [ 60%] Meshing curve 121 (Line)
Info    : [ 60%] Meshing curve 122 (Line)
Info    : [ 60%] Meshing curve 123 (Line)
Info    : 

false